<a href="https://colab.research.google.com/github/SaswatD27/SyrDataSuppression/blob/main/ACSAnalysisAug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import pipeline_dp as pdp
import math
import random
import data_tools_syr3 as syr
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics, tree 
from scipy import stats
import timeit
import xlwt
from xlwt import Workbook

In [ ]:
#prepare hist
def prep_hist(df):
  mar_df_1=df.groupby(['PUMA','AGE','RACE','SEX','OWNERSHP','INCTOT_DECILE'])[['GQ']]
  hist=mar_df_1.count()
  hist.reset_index()
  hist1=hist.copy()
  for i in range(6):
    hist1=hist1.reset_index(level=0)
  #print(hist1)
  hist1=hist1.rename(columns={'GQ':"COUNT"})
  return hist1
def prep_hist2(df,fnames):
  mar_df_1=df.groupby(fnames)[['INDNAICS']]
  hist=mar_df_1.count()
  hist.reset_index()
  hist1=hist.copy()
  for i in range(len(fnames)):
    hist1=hist1.reset_index(level=0)
  #print(hist1)
  hist1=hist1.rename(columns={'INDNAICS':"COUNT"})
  return hist1
def prep_hist3(df,fnames,cntname):
  mar_df_1=df.groupby(fnames)[[cntname]]
  hist=mar_df_1.count()
  hist.reset_index()
  hist1=hist.copy()
  for i in range(len(fnames)):
    hist1=hist1.reset_index(level=0)
  #print(hist1)
  hist1=hist1.rename(columns={cntname:"COUNT"})
  return hist1
#create function to replicate rows based on GQ value
def count_scale_df(hist):
  hist_r=hist.copy()
  for index, row in hist.iterrows():
    #print(row['COUNT'])
    for i in range(int(row['COUNT'])):
      hist_r=hist_r.append(row)
  return hist_r
#
def count_scale_df2(hist):
  hist_r=hist.copy()
  concat_rows=[]
  for index, row in hist.iterrows():
    #print(row['COUNT'])
    for i in range(int(row['COUNT'])):
      concat_rows.append(row)
  hist_r=pd.concat(concat_rows)
  return hist_r

def redact_counts_df(hist,k): #Outputs redact-suppressed histogram
  hist1=hist.copy()
  counts_1=list(hist1['COUNT'])
  print("counts:",counts_1)
  #print("Hola:",counts_1)
  for i in range(len(counts_1)):
    if counts_1[i]<k:
      counts_1[i]=(k//2)
  hist1['COUNT']=counts_1
  return hist1
#
def dp_counts_df(hist,eps): #Outputs redact-suppressed histogram
  hist1=hist.copy()
  counts_1=list(hist1['COUNT'])
  #try:
    #dp_counts=add_dp_hist(counts_1,eps)
  #except:
  dp_counts=syr.add_dp_hist(counts_1,eps)
  hist1['COUNT']=dp_counts
  return hist1
#
def randmerge_counts_df(hist,k): #Outputs redact-suppressed histogram
  hist1=hist.copy()
  #start=timeit.default_timer()
  counts_1=list(hist1['COUNT'])
  #try:
  dp_counts=syr.spread_k_anonymise(counts_1,k)
  #except:
    #dp_counts=syr.spread_k_anonymise(counts_1,k)
  hist1['COUNT']=dp_counts
  #stop=timeit.default_timer()
  #print('Time: ',stop-start)
  return hist1

def adjmerge_counts_df(hist,k):
  #print('hey')
  hist1=hist.copy()
  #start=timeit.default_timer()
  counts_1=list(hist1['COUNT'])
  #try:
  dp_counts=syr.adj_k_anonymise(counts_1,k)
  print(dp_counts)
  #except:
    #dp_counts=syr.spread_k_anonymise(counts_1,k)
  hist1['COUNT']=dp_counts
  #stop=timeit.default_timer()
  #print('Time: ',stop-start)
  return hist1

# DTs

In [ ]:
wbb=Workbook()

In [ ]:
#Get Adjmerge Stats
df1=pd.read_csv("tx_acs_2019.csv")
pumas=list(df1['PUMA'].unique())
ct=0
k=10
sheet2=wbb.add_sheet('TX - AM 10')
sheet3=wbb.add_sheet('TX Feat Imp - AM 10')
sheet2.write(0, 0, 'PUMAs')
sheet2.write(0, 1, 'Race')
sheet2.write(0, 2, 'Accuracy')
sheet2.write(0, 3, 'Height')
sheet2.write(0, 4, 'Opt Ht')
sheet2.write(0, 5, 'Opt Acc')
sheet2.write(0, 6, 'Max Ht 10')
sheet2.write(0, 7, 'Max Ht 20')
sheet2.write(0, 8, 'Pop %')
sheet3.write(0,0,'PUMAs')
sheet3.write(0,1,'Race')
sheet3.write(0,2,'GQ')
sheet3.write(0,3,'OWNERSHP')
sheet3.write(0,4,'FAMSIZE')
sheet3.write(0,5,'NCHILD')
sheet3.write(0,6,'AGE')
sheet3.write(0,7,'SEX')
sheet3.write(0,8,'MARST')
sheet3.write(0,9,'RACE')
sheet3.write(0,10,'HISPAN')
sheet3.write(0,11,'EDUC')
sheet3.write(0,12,'SECTOR')
sheet3.write(0,13,'POVERTY')
sheet3.write(0,14,'DENSITY')
sheet3.write(0,15,'VETDISAB')
sheet3.write(0,16,'DIFFREM')
sheet3.write(0,17,'DIFFPHYS')
sheet3.write(0,18,'DIFFEYE')
sheet3.write(0,19,'DIFFHEAR')
sheet3.write(0,20,'HHWT')
sheet3.write(0,21,'PERWT')
#pumas=[pumas[1]]
for puma in pumas:
  print('\n',puma)
  for race in range(1,8):
    print('\nRace - ',race)
    ct+=1
    sheet2.write(ct,0,puma)
    sheet2.write(ct,1,race)
    #print('\nRACE = ',race)
    #print(puma_1_df['COUNT'])
    accuracy=0
    height=0
    feature_importances_n=np.zeros(20)
    features_used=0
    #for _ in range(100):
    puma_1_df=df1
    puma_1_df=puma_1_df[(puma_1_df['PUMA']==puma)&(puma_1_df['RACE']==race)]
    #feature_cols=['OWNERSHP','FAMSIZE','NCHILD','AGE','SEX','RACE','EDUC','SECTOR']
    feature_cols=list(puma_1_df.columns)
    #print(feature_cols)
    feature_cols.remove('INCTOT')
    feature_cols.remove('PUMA')
    feature_cols.remove('INDNAICS')
    feature_cols.remove('Unnamed: 0')
    puma_1_df=prep_hist2(puma_1_df,feature_cols)
    if len(puma_1_df)<2:
      continue
    puma_1_df=adjmerge_counts_df(puma_1_df,k)
    puma_1_df=count_scale_df(puma_1_df)
    feature_cols.remove('INCTOT_DECILE')
    #print(puma_1_df.columns)
    X=puma_1_df[feature_cols]
    y=puma_1_df['INCTOT_DECILE']
    #print('\n',feature_cols)
    np.random.seed(0)
    try:
      X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
    except:
      #sheet2.write(ct,2,'Beep')
      #print('Not enough individuals')
      continue
    #print(y_test)
    #np.random.seed(0)
    dt_1=DecisionTreeClassifier(random_state=0)#, max_depth=2)
    dt_1=dt_1.fit(X_train,y_train)
    y_pred=dt_1.predict(X_test)
    accuracy=(metrics.accuracy_score(y_test,y_pred)*100)
    #print("Accuracy: ",metrics.accuracy_score(y_test,y_pred)*100)
    height+=(1+dt_1.tree_.max_depth)
    print(height)
    feature_importances = pd.DataFrame(dt_1.feature_importances_,index = X_train.columns,columns=['importance'])#.sort_values('importance', ascending=False)
    #print(feature_importances)
    #ctf=ct
    #rown=2
    #sheet3.write(ctf,0,puma)
    #sheet3.write(ctf,1,race)
    '''
    for row in feature_importances.index:
      sheet3.write(ctf,rown,feature_importances['importance'][row])
      rown+=1
    '''
    #feature_importances=np.array(feature_importances['importance'])
    features_used=(len(feature_importances[feature_importances['importance']>0]))
    sheet2.write(ct,2,accuracy)
    sheet2.write(ct,3,height)
    sheet2.write(ct,10,features_used)
    for rown in range(20):
      sheet3.write(ct,rown+2,feature_importances['importance'][rown])
    #accuracy for blacks
    #y_black_test=y_test[X_test['RACE']==2]
    #print(len(y_black_test))
    #X_black_test=X_test[X_test['RACE']==2]
    #print(len(X_black_test))
    #y_black_pred=dt_1.predict(X_black_test)
    #print('%age of Black People:',len(puma_1_df[(puma_1_df['RACE']==2)].index)/len(puma_1_df)*100)# & (puma_1_df['RACE']==2)]
    #print('%age of Black Women:',len(puma_1_df[(puma_1_df['RACE']==2)& (puma_1_df['SEX']==2)].index)/len(puma_1_df)*100)
    #print('%age of Non-White People:',len(puma_1_df[puma_1_df['RACE']!=1].index)/len(puma_1_df)*100)
    #print('%age of Women:',len(puma_1_df[puma_1_df['SEX']==2].index)/len(puma_1_df)*100)
    #print('%age of People > 60 yrs:',len(puma_1_df[(puma_1_df['AGE']>60)].index)/len(puma_1_df)*100)
    #print("Percentage = ",(len(puma_1_df_black.index)/len(puma_1_df)*100))

    max_d=dt_1.tree_.max_depth
    acc_list=[]
    for i in range(1,max_d+1):
      np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0, max_depth=i)
      dt_1=dt_1.fit(X_train,y_train)
      y_pred=dt_1.predict(X_test)
      acc_list.append(metrics.accuracy_score(y_test,y_pred)*100)
    try:
      sheet2.write(ct,4,2+acc_list.index(max(acc_list)))
      #print("Optimal Tree Height:",2+acc_list.index(max(acc_list)))
      #print("Max Accuracy:",max(acc_list))
      sheet2.write(ct,5,max(acc_list))
      np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0, max_depth=9)
      dt_1=dt_1.fit(X_train,y_train)
      #print(1+dt_1.tree_.max_depth)
      y_pred=dt_1.predict(X_test)
      sheet2.write(ct,6,(metrics.accuracy_score(y_test,y_pred)*100))
      #print('Accuracy for Max Height 10:',(metrics.accuracy_score(y_test,y_pred)*100))
      np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0, max_depth=19)
      dt_1=dt_1.fit(X_train,y_train)
      #print(1+dt_1.tree_.max_depth)
      y_pred=dt_1.predict(X_test)
      sheet2.write(ct,7,(metrics.accuracy_score(y_test,y_pred)*100))
      #print('Accuracy for Max Height 20:',(metrics.accuracy_score(y_test,y_pred)*100))
      sheet2.write(ct,8,round((len(df1[(df1['PUMA']==puma)&(df1['RACE']==race)])/len(df1[df1['PUMA']==puma]))*100,3))
    except:
      sheet2.write(ct,4,'Uh oh')
      continue
    #print("Accuracy for black rows is ",metrics.accuracy_score(y_black_test,y_black_pred)*100)
    #tree.plot_tree(dt_1);
wbb.save('RMforDTdedicated.xls')

Streaming output truncated to the last 5000 lines.
720
720
720
720
720
720
720
720
720
499
721
721
721
721
721
721
721
721
721
498
722
722
722
722
722
722
722
722
722
497
723
723
723
723
723
723
723
723
723
496
724
724
724
724
724
724
724
724
724
495
725
725
725
725
725
725
725
725
725
494
726
726
726
726
726
726
726
726
726
493
727
727
727
727
727
727
727
727
727
492
728
728
728
728
728
728
728
728
728
491
729
729
729
729
729
729
729
729
729
490
730
730
730
730
730
730
730
730
730
489
731
731
731
731
731
731
731
731
731
488
732
732
732
732
732
732
732
732
732
487
733
733
733
733
733
733
733
733
733
486
734
734
734
734
734
734
734
734
734
485
735
735
735
735
735
735
735
735
735
484
736
736
736
736
736
736
736
736
736
483
737
737
737
737
737
737
737
737
737
482
738
738
738
738
738
738
738
738
738
481
739
739
739
739
739
739
739
739
739
480
740
740
740
740
740
740
740
740
740
479
741
741
741
741
741
741
741
741
741
478
742
742
742
742
742
742
742
742
742
477
743
743
743
743
743
743
743
7

KeyError: ignored

In [ ]:
wbb=Workbook()

In [ ]:
#Get Adjmerge Stats; 4 features
df1=pd.read_csv("tx_acs_2019.csv")
pumas=list(df1['PUMA'].unique())
ct=0
k=10
sheet2=wbb.add_sheet('TX - Unperturbed')
sheet3=wbb.add_sheet('TX Feat Imp - Unperturbed')
sheet2.write(0, 0, 'PUMAs')
sheet2.write(0, 1, 'Race')
sheet2.write(0, 2, 'Accuracy')
sheet2.write(0, 3, 'Height')
sheet2.write(0, 4, 'Opt Ht')
sheet2.write(0, 5, 'Opt Acc')
sheet2.write(0, 6, 'Max Ht 10')
sheet2.write(0, 7, 'Max Ht 20')
sheet2.write(0, 8, 'Pop %')
sheet3.write(0,0,'PUMAs')
sheet3.write(0,1,'AGE')
sheet3.write(0,2,'SEX')
sheet3.write(0,3,'RACE')
sheet3.write(0,4,'OWNERSHP')
#pumas=[pumas[1]]
for puma in pumas:
  print('\n',puma)
  for race in range(1,8):
    print('\nRace - ',race)
    ct+=1
    sheet2.write(ct,0,puma)
    sheet2.write(ct,1,race)
    #print('\nRACE = ',race)
    #print(puma_1_df['COUNT'])
    accuracy=0
    height=0
    feature_importances_n=np.zeros(20)
    features_used=0
    #for _ in range(100):
    puma_1_df=df1
    puma_1_df=puma_1_df[(puma_1_df['PUMA']==puma)&(puma_1_df['RACE']==race)]
    #feature_cols=['OWNERSHP','FAMSIZE','NCHILD','AGE','SEX','RACE','EDUC','SECTOR']
    feature_cols=['AGE','SEX','RACE','OWNERSHP','INCTOT_DECILE']
    puma_1_df=prep_hist2(puma_1_df,feature_cols)
    if len(puma_1_df)<2:
      continue
    #puma_1_df=adjmerge_counts_df(puma_1_df,k)
    #puma_1_df=redact_counts_df(puma_1_df,k)
    puma_1_df=count_scale_df(puma_1_df)
    feature_cols.remove('INCTOT_DECILE')
    #print(puma_1_df.columns)
    X=puma_1_df[feature_cols]
    y=puma_1_df['INCTOT_DECILE']
    #print('\n',feature_cols)
    np.random.seed(0)
    try:
      X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
    except:
      #sheet2.write(ct,2,'Beep')
      #print('Not enough individuals')
      continue
    #print(y_test)
    #np.random.seed(0)
    dt_1=DecisionTreeClassifier(random_state=0)#, max_depth=2)
    dt_1=dt_1.fit(X_train,y_train)
    y_pred=dt_1.predict(X_test)
    accuracy=(metrics.accuracy_score(y_test,y_pred)*100)
    #print("Accuracy: ",metrics.accuracy_score(y_test,y_pred)*100)
    height+=(1+dt_1.tree_.max_depth)
    print(height)
    feature_importances = pd.DataFrame(dt_1.feature_importances_,index = X_train.columns,columns=['importance'])#.sort_values('importance', ascending=False)
    #print(feature_importances)
    #ctf=ct
    #rown=2
    #sheet3.write(ctf,0,puma)
    #sheet3.write(ctf,1,race)
    '''
    for row in feature_importances.index:
      sheet3.write(ctf,rown,feature_importances['importance'][row])
      rown+=1
    '''
    #feature_importances=np.array(feature_importances['importance'])
    features_used=(len(feature_importances[feature_importances['importance']>0]))
    sheet2.write(ct,2,accuracy)
    sheet2.write(ct,3,height)
    sheet2.write(ct,10,features_used)
    for rown in range(4):
      sheet3.write(ct,rown+2,feature_importances['importance'][rown])
    #accuracy for blacks
    #y_black_test=y_test[X_test['RACE']==2]
    #print(len(y_black_test))
    #X_black_test=X_test[X_test['RACE']==2]
    #print(len(X_black_test))
    #y_black_pred=dt_1.predict(X_black_test)
    #print('%age of Black People:',len(puma_1_df[(puma_1_df['RACE']==2)].index)/len(puma_1_df)*100)# & (puma_1_df['RACE']==2)]
    #print('%age of Black Women:',len(puma_1_df[(puma_1_df['RACE']==2)& (puma_1_df['SEX']==2)].index)/len(puma_1_df)*100)
    #print('%age of Non-White People:',len(puma_1_df[puma_1_df['RACE']!=1].index)/len(puma_1_df)*100)
    #print('%age of Women:',len(puma_1_df[puma_1_df['SEX']==2].index)/len(puma_1_df)*100)
    #print('%age of People > 60 yrs:',len(puma_1_df[(puma_1_df['AGE']>60)].index)/len(puma_1_df)*100)
    #print("Percentage = ",(len(puma_1_df_black.index)/len(puma_1_df)*100))

    max_d=dt_1.tree_.max_depth
    acc_list=[]
    for i in range(1,max_d+1):
      np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0, max_depth=i)
      dt_1=dt_1.fit(X_train,y_train)
      y_pred=dt_1.predict(X_test)
      acc_list.append(metrics.accuracy_score(y_test,y_pred)*100)
    try:
      sheet2.write(ct,4,2+acc_list.index(max(acc_list)))
      #print("Optimal Tree Height:",2+acc_list.index(max(acc_list)))
      #print("Max Accuracy:",max(acc_list))
      sheet2.write(ct,5,max(acc_list))
      np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0, max_depth=9)
      dt_1=dt_1.fit(X_train,y_train)
      #print(1+dt_1.tree_.max_depth)
      y_pred=dt_1.predict(X_test)
      sheet2.write(ct,6,(metrics.accuracy_score(y_test,y_pred)*100))
      #print('Accuracy for Max Height 10:',(metrics.accuracy_score(y_test,y_pred)*100))
      np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0, max_depth=19)
      dt_1=dt_1.fit(X_train,y_train)
      #print(1+dt_1.tree_.max_depth)
      y_pred=dt_1.predict(X_test)
      sheet2.write(ct,7,(metrics.accuracy_score(y_test,y_pred)*100))
      #print('Accuracy for Max Height 20:',(metrics.accuracy_score(y_test,y_pred)*100))
      sheet2.write(ct,8,round((len(df1[(df1['PUMA']==puma)&(df1['RACE']==race)])/len(df1[df1['PUMA']==puma]))*100,3))
    except:
      sheet2.write(ct,4,'Uh oh')
      continue
    #print("Accuracy for black rows is ",metrics.accuracy_score(y_black_test,y_black_pred)*100)
    #tree.plot_tree(dt_1);
wbb.save('RMforDTdedicated4f.xls')


 48-2510

Race -  1
21

Race -  2
11

Race -  3
8

Race -  4
6

Race -  5

Race -  6
15

Race -  7
10

 48-2102

Race -  1
19

Race -  2
11

Race -  3
5

Race -  4

Race -  5

Race -  6
7

Race -  7
9

 48-2101

Race -  1
22

Race -  2
16

Race -  3
7

Race -  4

Race -  5

Race -  6
10

Race -  7
12

 48-2515

Race -  1
20

Race -  2
14

Race -  3
3

Race -  4
3

Race -  5

Race -  6
9

Race -  7
13

 48-2507

Race -  1
21

Race -  2
16

Race -  3
2

Race -  4
6

Race -  5
2

Race -  6
9

Race -  7
12

 48-2516

Race -  1
18

Race -  2
12

Race -  3
6

Race -  4
5

Race -  5

Race -  6
10

Race -  7
8


In [ ]:
wbb=Workbook()

In [ ]:
#Get Adjmerge Stats; 4 features; randomised processes
wbb=Workbook()
dfname="tx_acs_2019.csv"
#RM 6
df1=pd.read_csv(dfname)
pumas=list(df1['PUMA'].unique())
ct=0
k=6
sheet2=wbb.add_sheet('TX - RM 6')
sheet3=wbb.add_sheet('TX Feat Imp - RM 6')
sheet2.write(0, 0, 'PUMAs')
sheet2.write(0, 1, 'Race')
sheet2.write(0, 2, 'Accuracy')
sheet2.write(0, 3, 'Height')
sheet2.write(0, 4, 'Opt Ht')
sheet2.write(0, 5, 'Opt Acc')
sheet2.write(0, 6, 'Max Ht 10')
sheet2.write(0, 7, 'Max Ht 20')
sheet2.write(0, 8, 'Pop %')
sheet3.write(0,0,'PUMAs')
sheet3.write(0,1,'AGE')
sheet3.write(0,2,'SEX')
sheet3.write(0,3,'RACE')
sheet3.write(0,4,'OWNERSHP')
#pumas=[pumas[1]]
for puma in pumas:
  print('\n',puma)
  for race in range(1,8):
    print('\nRace - ',race)
    ct+=1
    sheet2.write(ct,0,puma)
    sheet2.write(ct,1,race)
    #print('\nRACE = ',race)
    #print(puma_1_df['COUNT'])
    accuracy=0
    height=0
    feature_importances_n=np.zeros(4)
    features_used=0
    opt_ht=0
    opt_acc=0
    ht_10=0
    ht_20=0
    for _ in range(100):
      puma_1_df=df1.copy()
      puma_1_df=puma_1_df[(puma_1_df['PUMA']==puma)&(puma_1_df['RACE']==race)]
      #feature_cols=['OWNERSHP','FAMSIZE','NCHILD','AGE','SEX','RACE','EDUC','SECTOR']
      feature_cols=['AGE','SEX','RACE','OWNERSHP','INCTOT_DECILE']
      puma_1_df=prep_hist2(puma_1_df,feature_cols)
      if len(puma_1_df)<2:
        continue
      puma_1_df=randmerge_counts_df(puma_1_df,k)
      #puma_1_df=dp_counts_df(puma_1_df,k)
      puma_1_df=count_scale_df(puma_1_df)
      feature_cols.remove('INCTOT_DECILE')
      #print(puma_1_df.columns)
      X=puma_1_df[feature_cols]
      y=puma_1_df['INCTOT_DECILE']
      #print('\n',feature_cols)
      np.random.seed(0)
      try:
        X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
      except:
        #sheet2.write(ct,2,'Beep')
        #print('Not enough individuals')
        continue
      #print(y_test)
      #np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0)#, max_depth=2)
      dt_1=dt_1.fit(X_train,y_train)
      y_pred=dt_1.predict(X_test)
      accuracy+=(metrics.accuracy_score(y_test,y_pred)*100)
      #print("Accuracy: ",metrics.accuracy_score(y_test,y_pred)*100)
      height+=(1+dt_1.tree_.max_depth)
      print(height)
      feature_importances = pd.DataFrame(dt_1.feature_importances_,index = X_train.columns,columns=['importance'])#.sort_values('importance', ascending=False)
      feature_importances_n+=np.array(feature_importances['importance'])
      #print(feature_importances)
      #ctf=ct
      #rown=2
      #sheet3.write(ctf,0,puma)
      #sheet3.write(ctf,1,race)
      '''
      for row in feature_importances.index:
        sheet3.write(ctf,rown,feature_importances['importance'][row])
        rown+=1
      '''
      #feature_importances=np.array(feature_importances['importance'])
      features_used+=(len(feature_importances[feature_importances['importance']>0]))
      #accuracy for blacks
      #y_black_test=y_test[X_test['RACE']==2]
      #print(len(y_black_test))
      #X_black_test=X_test[X_test['RACE']==2]
      #print(len(X_black_test))
      #y_black_pred=dt_1.predict(X_black_test)
      #print('%age of Black People:',len(puma_1_df[(puma_1_df['RACE']==2)].index)/len(puma_1_df)*100)# & (puma_1_df['RACE']==2)]
      #print('%age of Black Women:',len(puma_1_df[(puma_1_df['RACE']==2)& (puma_1_df['SEX']==2)].index)/len(puma_1_df)*100)
      #print('%age of Non-White People:',len(puma_1_df[puma_1_df['RACE']!=1].index)/len(puma_1_df)*100)
      #print('%age of Women:',len(puma_1_df[puma_1_df['SEX']==2].index)/len(puma_1_df)*100)
      #print('%age of People > 60 yrs:',len(puma_1_df[(puma_1_df['AGE']>60)].index)/len(puma_1_df)*100)
      #print("Percentage = ",(len(puma_1_df_black.index)/len(puma_1_df)*100))
      max_d=dt_1.tree_.max_depth
      acc_list=[]
      for i in range(1,max_d+1):
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=i)
        dt_1=dt_1.fit(X_train,y_train)
        y_pred=dt_1.predict(X_test)
        acc_list.append(metrics.accuracy_score(y_test,y_pred)*100)
      try:
        opt_ht+=2+acc_list.index(max(acc_list))
        #print("Optimal Tree Height:",2+acc_list.index(max(acc_list)))
        #print("Max Accuracy:",max(acc_list))
        opt_acc+=max(acc_list)
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=9)
        dt_1=dt_1.fit(X_train,y_train)
        #print(1+dt_1.tree_.max_depth)
        y_pred=dt_1.predict(X_test)
        ht_10+=(metrics.accuracy_score(y_test,y_pred)*100)
        #print('Accuracy for Max Height 10:',(metrics.accuracy_score(y_test,y_pred)*100))
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=19)
        dt_1=dt_1.fit(X_train,y_train)
        #print(1+dt_1.tree_.max_depth)
        y_pred=dt_1.predict(X_test)
        ht_20+=(metrics.accuracy_score(y_test,y_pred)*100)
        #print('Accuracy for Max Height 20:',(metrics.accuracy_score(y_test,y_pred)*100))
      except:
        sheet2.write(ct,4,'Uh oh')
        continue
    accuracy/=100
    height/=100
    feature_importances_n/=100
    features_used/=100
    opt_ht/=100
    opt_acc/=100
    ht_10/=100
    ht_20/=100
    sheet2.write(ct,2,accuracy)
    sheet2.write(ct,3,height)
    sheet2.write(ct,10,features_used)
    sheet2.write(ct,8,round((len(df1[(df1['PUMA']==puma)&(df1['RACE']==race)])/len(df1[df1['PUMA']==puma]))*100,3))
    for rown in range(4):
      sheet3.write(ct,rown+2,feature_importances['importance'][rown])
    sheet2.write(ct, 4, opt_ht) 
    sheet2.write(ct, 5, opt_acc)
    sheet2.write(ct, 6, ht_10) 
    sheet2.write(ct, 7, ht_20)  
    #print("Accuracy for black rows is ",metrics.accuracy_score(y_black_test,y_black_pred)*100)
    #tree.plot_tree(dt_1);
wbb.save('RMforDTdedicated4frand.xls')
#RM 10
df1=pd.read_csv(dfname)
pumas=list(df1['PUMA'].unique())
ct=0
k=10
sheet2=wbb.add_sheet('TX - RM 10')
sheet3=wbb.add_sheet('TX Feat Imp - RM 10')
sheet2.write(0, 0, 'PUMAs')
sheet2.write(0, 1, 'Race')
sheet2.write(0, 2, 'Accuracy')
sheet2.write(0, 3, 'Height')
sheet2.write(0, 4, 'Opt Ht')
sheet2.write(0, 5, 'Opt Acc')
sheet2.write(0, 6, 'Max Ht 10')
sheet2.write(0, 7, 'Max Ht 20')
sheet2.write(0, 8, 'Pop %')
sheet3.write(0,0,'PUMAs')
sheet3.write(0,1,'AGE')
sheet3.write(0,2,'SEX')
sheet3.write(0,3,'RACE')
sheet3.write(0,4,'OWNERSHP')
#pumas=[pumas[1]]
for puma in pumas:
  print('\n',puma)
  for race in range(1,8):
    print('\nRace - ',race)
    ct+=1
    sheet2.write(ct,0,puma)
    sheet2.write(ct,1,race)
    #print('\nRACE = ',race)
    #print(puma_1_df['COUNT'])
    accuracy=0
    height=0
    feature_importances_n=np.zeros(4)
    features_used=0
    opt_ht=0
    opt_acc=0
    ht_10=0
    ht_20=0
    for _ in range(100):
      puma_1_df=df1.copy()
      puma_1_df=puma_1_df[(puma_1_df['PUMA']==puma)&(puma_1_df['RACE']==race)]
      #feature_cols=['OWNERSHP','FAMSIZE','NCHILD','AGE','SEX','RACE','EDUC','SECTOR']
      feature_cols=['AGE','SEX','RACE','OWNERSHP','INCTOT_DECILE']
      puma_1_df=prep_hist2(puma_1_df,feature_cols)
      if len(puma_1_df)<2:
        continue
      puma_1_df=randmerge_counts_df(puma_1_df,k)
      #puma_1_df=dp_counts_df(puma_1_df,k)
      puma_1_df=count_scale_df(puma_1_df)
      feature_cols.remove('INCTOT_DECILE')
      #print(puma_1_df.columns)
      X=puma_1_df[feature_cols]
      y=puma_1_df['INCTOT_DECILE']
      #print('\n',feature_cols)
      np.random.seed(0)
      try:
        X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
      except:
        #sheet2.write(ct,2,'Beep')
        #print('Not enough individuals')
        continue
      #print(y_test)
      #np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0)#, max_depth=2)
      dt_1=dt_1.fit(X_train,y_train)
      y_pred=dt_1.predict(X_test)
      accuracy+=(metrics.accuracy_score(y_test,y_pred)*100)
      #print("Accuracy: ",metrics.accuracy_score(y_test,y_pred)*100)
      height+=(1+dt_1.tree_.max_depth)
      print(height)
      feature_importances = pd.DataFrame(dt_1.feature_importances_,index = X_train.columns,columns=['importance'])#.sort_values('importance', ascending=False)
      feature_importances_n+=np.array(feature_importances['importance'])
      #print(feature_importances)
      #ctf=ct
      #rown=2
      #sheet3.write(ctf,0,puma)
      #sheet3.write(ctf,1,race)
      '''
      for row in feature_importances.index:
        sheet3.write(ctf,rown,feature_importances['importance'][row])
        rown+=1
      '''
      #feature_importances=np.array(feature_importances['importance'])
      features_used+=(len(feature_importances[feature_importances['importance']>0]))
      #accuracy for blacks
      #y_black_test=y_test[X_test['RACE']==2]
      #print(len(y_black_test))
      #X_black_test=X_test[X_test['RACE']==2]
      #print(len(X_black_test))
      #y_black_pred=dt_1.predict(X_black_test)
      #print('%age of Black People:',len(puma_1_df[(puma_1_df['RACE']==2)].index)/len(puma_1_df)*100)# & (puma_1_df['RACE']==2)]
      #print('%age of Black Women:',len(puma_1_df[(puma_1_df['RACE']==2)& (puma_1_df['SEX']==2)].index)/len(puma_1_df)*100)
      #print('%age of Non-White People:',len(puma_1_df[puma_1_df['RACE']!=1].index)/len(puma_1_df)*100)
      #print('%age of Women:',len(puma_1_df[puma_1_df['SEX']==2].index)/len(puma_1_df)*100)
      #print('%age of People > 60 yrs:',len(puma_1_df[(puma_1_df['AGE']>60)].index)/len(puma_1_df)*100)
      #print("Percentage = ",(len(puma_1_df_black.index)/len(puma_1_df)*100))
      max_d=dt_1.tree_.max_depth
      acc_list=[]
      for i in range(1,max_d+1):
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=i)
        dt_1=dt_1.fit(X_train,y_train)
        y_pred=dt_1.predict(X_test)
        acc_list.append(metrics.accuracy_score(y_test,y_pred)*100)
      try:
        opt_ht+=2+acc_list.index(max(acc_list))
        #print("Optimal Tree Height:",2+acc_list.index(max(acc_list)))
        #print("Max Accuracy:",max(acc_list))
        opt_acc+=max(acc_list)
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=9)
        dt_1=dt_1.fit(X_train,y_train)
        #print(1+dt_1.tree_.max_depth)
        y_pred=dt_1.predict(X_test)
        ht_10+=(metrics.accuracy_score(y_test,y_pred)*100)
        #print('Accuracy for Max Height 10:',(metrics.accuracy_score(y_test,y_pred)*100))
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=19)
        dt_1=dt_1.fit(X_train,y_train)
        #print(1+dt_1.tree_.max_depth)
        y_pred=dt_1.predict(X_test)
        ht_20+=(metrics.accuracy_score(y_test,y_pred)*100)
        #print('Accuracy for Max Height 20:',(metrics.accuracy_score(y_test,y_pred)*100))
      except:
        sheet2.write(ct,4,'Uh oh')
        continue
    accuracy/=100
    height/=100
    feature_importances_n/=100
    features_used/=100
    opt_ht/=100
    opt_acc/=100
    ht_10/=100
    ht_20/=100
    sheet2.write(ct,2,accuracy)
    sheet2.write(ct,3,height)
    sheet2.write(ct,10,features_used)
    sheet2.write(ct,8,round((len(df1[(df1['PUMA']==puma)&(df1['RACE']==race)])/len(df1[df1['PUMA']==puma]))*100,3))
    for rown in range(4):
      sheet3.write(ct,rown+2,feature_importances['importance'][rown])
    sheet2.write(ct, 4, opt_ht) 
    sheet2.write(ct, 5, opt_acc)
    sheet2.write(ct, 6, ht_10) 
    sheet2.write(ct, 7, ht_20)  
    #print("Accuracy for black rows is ",metrics.accuracy_score(y_black_test,y_black_pred)*100)
    #tree.plot_tree(dt_1);
wbb.save('RMforDTdedicated4frand.xls')
#DP 1
df1=pd.read_csv(dfname)
pumas=list(df1['PUMA'].unique())
ct=0
k=1
sheet2=wbb.add_sheet('TX - DP 1')
sheet3=wbb.add_sheet('TX Feat Imp - DP 1')
sheet2.write(0, 0, 'PUMAs')
sheet2.write(0, 1, 'Race')
sheet2.write(0, 2, 'Accuracy')
sheet2.write(0, 3, 'Height')
sheet2.write(0, 4, 'Opt Ht')
sheet2.write(0, 5, 'Opt Acc')
sheet2.write(0, 6, 'Max Ht 10')
sheet2.write(0, 7, 'Max Ht 20')
sheet2.write(0, 8, 'Pop %')
sheet3.write(0,0,'PUMAs')
sheet3.write(0,1,'AGE')
sheet3.write(0,2,'SEX')
sheet3.write(0,3,'RACE')
sheet3.write(0,4,'OWNERSHP')
#pumas=[pumas[1]]
for puma in pumas:
  print('\n',puma)
  for race in range(1,8):
    print('\nRace - ',race)
    ct+=1
    sheet2.write(ct,0,puma)
    sheet2.write(ct,1,race)
    #print('\nRACE = ',race)
    #print(puma_1_df['COUNT'])
    accuracy=0
    height=0
    feature_importances_n=np.zeros(4)
    features_used=0
    opt_ht=0
    opt_acc=0
    ht_10=0
    ht_20=0
    for _ in range(100):
      puma_1_df=df1.copy()
      puma_1_df=puma_1_df[(puma_1_df['PUMA']==puma)&(puma_1_df['RACE']==race)]
      #feature_cols=['OWNERSHP','FAMSIZE','NCHILD','AGE','SEX','RACE','EDUC','SECTOR']
      feature_cols=['AGE','SEX','RACE','OWNERSHP','INCTOT_DECILE']
      puma_1_df=prep_hist2(puma_1_df,feature_cols)
      if len(puma_1_df)<2:
        continue
      #puma_1_df=randmerge_counts_df(puma_1_df,k)
      puma_1_df=dp_counts_df(puma_1_df,k)
      puma_1_df=count_scale_df(puma_1_df)
      feature_cols.remove('INCTOT_DECILE')
      #print(puma_1_df.columns)
      X=puma_1_df[feature_cols]
      y=puma_1_df['INCTOT_DECILE']
      #print('\n',feature_cols)
      np.random.seed(0)
      try:
        X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
      except:
        #sheet2.write(ct,2,'Beep')
        #print('Not enough individuals')
        continue
      #print(y_test)
      #np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0)#, max_depth=2)
      dt_1=dt_1.fit(X_train,y_train)
      y_pred=dt_1.predict(X_test)
      accuracy+=(metrics.accuracy_score(y_test,y_pred)*100)
      #print("Accuracy: ",metrics.accuracy_score(y_test,y_pred)*100)
      height+=(1+dt_1.tree_.max_depth)
      print(height)
      feature_importances = pd.DataFrame(dt_1.feature_importances_,index = X_train.columns,columns=['importance'])#.sort_values('importance', ascending=False)
      feature_importances_n+=np.array(feature_importances['importance'])
      #print(feature_importances)
      #ctf=ct
      #rown=2
      #sheet3.write(ctf,0,puma)
      #sheet3.write(ctf,1,race)
      '''
      for row in feature_importances.index:
        sheet3.write(ctf,rown,feature_importances['importance'][row])
        rown+=1
      '''
      #feature_importances=np.array(feature_importances['importance'])
      features_used+=(len(feature_importances[feature_importances['importance']>0]))
      #accuracy for blacks
      #y_black_test=y_test[X_test['RACE']==2]
      #print(len(y_black_test))
      #X_black_test=X_test[X_test['RACE']==2]
      #print(len(X_black_test))
      #y_black_pred=dt_1.predict(X_black_test)
      #print('%age of Black People:',len(puma_1_df[(puma_1_df['RACE']==2)].index)/len(puma_1_df)*100)# & (puma_1_df['RACE']==2)]
      #print('%age of Black Women:',len(puma_1_df[(puma_1_df['RACE']==2)& (puma_1_df['SEX']==2)].index)/len(puma_1_df)*100)
      #print('%age of Non-White People:',len(puma_1_df[puma_1_df['RACE']!=1].index)/len(puma_1_df)*100)
      #print('%age of Women:',len(puma_1_df[puma_1_df['SEX']==2].index)/len(puma_1_df)*100)
      #print('%age of People > 60 yrs:',len(puma_1_df[(puma_1_df['AGE']>60)].index)/len(puma_1_df)*100)
      #print("Percentage = ",(len(puma_1_df_black.index)/len(puma_1_df)*100))
      max_d=dt_1.tree_.max_depth
      acc_list=[]
      for i in range(1,max_d+1):
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=i)
        dt_1=dt_1.fit(X_train,y_train)
        y_pred=dt_1.predict(X_test)
        acc_list.append(metrics.accuracy_score(y_test,y_pred)*100)
      try:
        opt_ht+=2+acc_list.index(max(acc_list))
        #print("Optimal Tree Height:",2+acc_list.index(max(acc_list)))
        #print("Max Accuracy:",max(acc_list))
        opt_acc+=max(acc_list)
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=9)
        dt_1=dt_1.fit(X_train,y_train)
        #print(1+dt_1.tree_.max_depth)
        y_pred=dt_1.predict(X_test)
        ht_10+=(metrics.accuracy_score(y_test,y_pred)*100)
        #print('Accuracy for Max Height 10:',(metrics.accuracy_score(y_test,y_pred)*100))
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=19)
        dt_1=dt_1.fit(X_train,y_train)
        #print(1+dt_1.tree_.max_depth)
        y_pred=dt_1.predict(X_test)
        ht_20+=(metrics.accuracy_score(y_test,y_pred)*100)
        #print('Accuracy for Max Height 20:',(metrics.accuracy_score(y_test,y_pred)*100))
      except:
        sheet2.write(ct,4,'Uh oh')
        continue
    accuracy/=100
    height/=100
    feature_importances_n/=100
    features_used/=100
    opt_ht/=100
    opt_acc/=100
    ht_10/=100
    ht_20/=100
    sheet2.write(ct,2,accuracy)
    sheet2.write(ct,3,height)
    sheet2.write(ct,10,features_used)
    sheet2.write(ct,8,round((len(df1[(df1['PUMA']==puma)&(df1['RACE']==race)])/len(df1[df1['PUMA']==puma]))*100,3))
    for rown in range(4):
      sheet3.write(ct,rown+2,feature_importances['importance'][rown])
    sheet2.write(ct, 4, opt_ht) 
    sheet2.write(ct, 5, opt_acc)
    sheet2.write(ct, 6, ht_10) 
    sheet2.write(ct, 7, ht_20)  
    #print("Accuracy for black rows is ",metrics.accuracy_score(y_black_test,y_black_pred)*100)
    #tree.plot_tree(dt_1);
wbb.save('RMforDTdedicated4frand.xls')
#DP 1
df1=pd.read_csv(dfname)
pumas=list(df1['PUMA'].unique())
ct=0
k=2
sheet2=wbb.add_sheet('TX - DP 2')
sheet3=wbb.add_sheet('TX Feat Imp - DP 2')
sheet2.write(0, 0, 'PUMAs')
sheet2.write(0, 1, 'Race')
sheet2.write(0, 2, 'Accuracy')
sheet2.write(0, 3, 'Height')
sheet2.write(0, 4, 'Opt Ht')
sheet2.write(0, 5, 'Opt Acc')
sheet2.write(0, 6, 'Max Ht 10')
sheet2.write(0, 7, 'Max Ht 20')
sheet2.write(0, 8, 'Pop %')
sheet3.write(0,0,'PUMAs')
sheet3.write(0,1,'AGE')
sheet3.write(0,2,'SEX')
sheet3.write(0,3,'RACE')
sheet3.write(0,4,'OWNERSHP')
#pumas=[pumas[1]]
for puma in pumas:
  print('\n',puma)
  for race in range(1,8):
    print('\nRace - ',race)
    ct+=1
    sheet2.write(ct,0,puma)
    sheet2.write(ct,1,race)
    #print('\nRACE = ',race)
    #print(puma_1_df['COUNT'])
    accuracy=0
    height=0
    feature_importances_n=np.zeros(4)
    features_used=0
    opt_ht=0
    opt_acc=0
    ht_10=0
    ht_20=0
    for _ in range(100):
      puma_1_df=df1.copy()
      puma_1_df=puma_1_df[(puma_1_df['PUMA']==puma)&(puma_1_df['RACE']==race)]
      #feature_cols=['OWNERSHP','FAMSIZE','NCHILD','AGE','SEX','RACE','EDUC','SECTOR']
      feature_cols=['AGE','SEX','RACE','OWNERSHP','INCTOT_DECILE']
      puma_1_df=prep_hist2(puma_1_df,feature_cols)
      if len(puma_1_df)<2:
        continue
      #puma_1_df=randmerge_counts_df(puma_1_df,k)
      puma_1_df=dp_counts_df(puma_1_df,k)
      puma_1_df=count_scale_df(puma_1_df)
      feature_cols.remove('INCTOT_DECILE')
      #print(puma_1_df.columns)
      X=puma_1_df[feature_cols]
      y=puma_1_df['INCTOT_DECILE']
      #print('\n',feature_cols)
      np.random.seed(0)
      try:
        X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
      except:
        #sheet2.write(ct,2,'Beep')
        #print('Not enough individuals')
        continue
      #print(y_test)
      #np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0)#, max_depth=2)
      dt_1=dt_1.fit(X_train,y_train)
      y_pred=dt_1.predict(X_test)
      accuracy+=(metrics.accuracy_score(y_test,y_pred)*100)
      #print("Accuracy: ",metrics.accuracy_score(y_test,y_pred)*100)
      height+=(1+dt_1.tree_.max_depth)
      print(height)
      feature_importances = pd.DataFrame(dt_1.feature_importances_,index = X_train.columns,columns=['importance'])#.sort_values('importance', ascending=False)
      feature_importances_n+=np.array(feature_importances['importance'])
      #print(feature_importances)
      #ctf=ct
      #rown=2
      #sheet3.write(ctf,0,puma)
      #sheet3.write(ctf,1,race)
      '''
      for row in feature_importances.index:
        sheet3.write(ctf,rown,feature_importances['importance'][row])
        rown+=1
      '''
      #feature_importances=np.array(feature_importances['importance'])
      features_used+=(len(feature_importances[feature_importances['importance']>0]))
      #accuracy for blacks
      #y_black_test=y_test[X_test['RACE']==2]
      #print(len(y_black_test))
      #X_black_test=X_test[X_test['RACE']==2]
      #print(len(X_black_test))
      #y_black_pred=dt_1.predict(X_black_test)
      #print('%age of Black People:',len(puma_1_df[(puma_1_df['RACE']==2)].index)/len(puma_1_df)*100)# & (puma_1_df['RACE']==2)]
      #print('%age of Black Women:',len(puma_1_df[(puma_1_df['RACE']==2)& (puma_1_df['SEX']==2)].index)/len(puma_1_df)*100)
      #print('%age of Non-White People:',len(puma_1_df[puma_1_df['RACE']!=1].index)/len(puma_1_df)*100)
      #print('%age of Women:',len(puma_1_df[puma_1_df['SEX']==2].index)/len(puma_1_df)*100)
      #print('%age of People > 60 yrs:',len(puma_1_df[(puma_1_df['AGE']>60)].index)/len(puma_1_df)*100)
      #print("Percentage = ",(len(puma_1_df_black.index)/len(puma_1_df)*100))
      max_d=dt_1.tree_.max_depth
      acc_list=[]
      for i in range(1,max_d+1):
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=i)
        dt_1=dt_1.fit(X_train,y_train)
        y_pred=dt_1.predict(X_test)
        acc_list.append(metrics.accuracy_score(y_test,y_pred)*100)
      try:
        opt_ht+=2+acc_list.index(max(acc_list))
        #print("Optimal Tree Height:",2+acc_list.index(max(acc_list)))
        #print("Max Accuracy:",max(acc_list))
        opt_acc+=max(acc_list)
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=9)
        dt_1=dt_1.fit(X_train,y_train)
        #print(1+dt_1.tree_.max_depth)
        y_pred=dt_1.predict(X_test)
        ht_10+=(metrics.accuracy_score(y_test,y_pred)*100)
        #print('Accuracy for Max Height 10:',(metrics.accuracy_score(y_test,y_pred)*100))
        np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0, max_depth=19)
        dt_1=dt_1.fit(X_train,y_train)
        #print(1+dt_1.tree_.max_depth)
        y_pred=dt_1.predict(X_test)
        ht_20+=(metrics.accuracy_score(y_test,y_pred)*100)
        #print('Accuracy for Max Height 20:',(metrics.accuracy_score(y_test,y_pred)*100))
      except:
        sheet2.write(ct,4,'Uh oh')
        continue
    accuracy/=100
    height/=100
    feature_importances_n/=100
    features_used/=100
    opt_ht/=100
    opt_acc/=100
    ht_10/=100
    ht_20/=100
    sheet2.write(ct,2,accuracy)
    sheet2.write(ct,3,height)
    sheet2.write(ct,10,features_used)
    sheet2.write(ct,8,round((len(df1[(df1['PUMA']==puma)&(df1['RACE']==race)])/len(df1[df1['PUMA']==puma]))*100,3))
    for rown in range(4):
      sheet3.write(ct,rown+2,feature_importances['importance'][rown])
    sheet2.write(ct, 4, opt_ht) 
    sheet2.write(ct, 5, opt_acc)
    sheet2.write(ct, 6, ht_10) 
    sheet2.write(ct, 7, ht_20)  
    #print("Accuracy for black rows is ",metrics.accuracy_score(y_black_test,y_black_pred)*100)
    #tree.plot_tree(dt_1);
wbb.save('RMforDTdedicated4frand.xls')


 48-2510

Race -  1
641
639
635
633
628
625
622
619
617
612
610
605
603
600
598
596
594
589
584
580
576
572
570
564
562
559
554
549
548
546
543
539
535
533
529
525
522
519
514
511
509
506
501
497
495
492
488
483
481
478
475
471
469
465
460
457
455
451
449
446
442
440
437
434
430
425
422
419
417
413
409
405
403
401
396
394
392
389
387
385
383
381
377
375
373
370
368
366
364
358
357
354
352
349
346
343
339
338
335
333
331
326
323
319
316
313
310
308
306
301
298
294
293
290
287
285
283
280
277
275
273
270
268
265
262
260
259
258
255
253
252
249
247
246
244
241
239
235
233
232
230
228
223
221
218
217
216
215
213
211
209
207
205
202
201
199
197
194
192
191
189
187
184
183
181
179
177
175
172
169
167
165
164
163
161
158
157
155
154
153
152
150
149
147
145
143
141
140
137
134
132
130
127
125
123
122
120
119
117
115
114
112
111
110
109
107
105
103
102
99
97
95
94
93
92
91
90
89
87
85
84
83
82
80
79
77
76
75
74
73
72
71
70
69
68
67
66
63
62
61
59
57
55
54
53
51
50
49
48
47
46
44
43
41
39
38
37

KeyboardInterrupt: ignored

In [ ]:
#Get Adjmerge Stats
df1=pd.read_csv("tx_acs_2019.csv")
pumas=list(df1['PUMA'].unique())
ct=0
k=
sheet2=wbb.add_sheet('TX - AM 10')
sheet3=wbb.add_sheet('TX Feat Imp - AM 10')
sheet2.write(0, 0, 'PUMAs')
sheet2.write(0, 1, 'Race')
sheet2.write(0, 2, 'Accuracy')
sheet2.write(0, 3, 'Height')
sheet2.write(0, 4, 'Opt Ht')
sheet2.write(0, 5, 'Opt Acc')
sheet2.write(0, 6, 'Max Ht 10')
sheet2.write(0, 7, 'Max Ht 20')
sheet2.write(0, 8, 'Pop %')
sheet3.write(0,0,'PUMAs')
sheet3.write(0,1,'Race')
sheet3.write(0,2,'GQ')
sheet3.write(0,3,'OWNERSHP')
sheet3.write(0,4,'FAMSIZE')
sheet3.write(0,5,'NCHILD')
sheet3.write(0,6,'AGE')
sheet3.write(0,7,'SEX')
sheet3.write(0,8,'MARST')
sheet3.write(0,9,'RACE')
sheet3.write(0,10,'HISPAN')
sheet3.write(0,11,'EDUC')
sheet3.write(0,12,'SECTOR')
sheet3.write(0,13,'POVERTY')
sheet3.write(0,14,'DENSITY')
sheet3.write(0,15,'VETDISAB')
sheet3.write(0,16,'DIFFREM')
sheet3.write(0,17,'DIFFPHYS')
sheet3.write(0,18,'DIFFEYE')
sheet3.write(0,19,'DIFFHEAR')
sheet3.write(0,20,'HHWT')
sheet3.write(0,21,'PERWT')
#pumas=[pumas[1]]
for puma in pumas:
  print('\n',puma)
  for race in range(1,8):
    print('\nRace - ',race)
    ct+=1
    sheet2.write(ct,0,puma)
    sheet2.write(ct,1,race)
    #print('\nRACE = ',race)
    #print(puma_1_df['COUNT'])
    accuracy=0
    height=0
    feature_importances_n=np.zeros(20)
    features_used=0
    #for _ in range(100):
    puma_1_df=df1
    puma_1_df=puma_1_df[(puma_1_df['PUMA']==puma)&(puma_1_df['RACE']==race)]
    #feature_cols=['OWNERSHP','FAMSIZE','NCHILD','AGE','SEX','RACE','EDUC','SECTOR']
    feature_cols=list(puma_1_df.columns)
    #print(feature_cols)
    feature_cols.remove('INCTOT')
    feature_cols.remove('PUMA')
    feature_cols.remove('INDNAICS')
    feature_cols.remove('Unnamed: 0')
    puma_1_df=prep_hist2(puma_1_df,feature_cols)
    if len(puma_1_df)<2:
      continue
    puma_1_df=adjmerge_counts_df(puma_1_df,k)
    puma_1_df=count_scale_df(puma_1_df)
    feature_cols.remove('INCTOT_DECILE')
    #print(puma_1_df.columns)
    X=puma_1_df[feature_cols]
    y=puma_1_df['INCTOT_DECILE']
    #print('\n',feature_cols)
    np.random.seed(0)
    try:
      X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
    except:
      #sheet2.write(ct,2,'Beep')
      #print('Not enough individuals')
      continue
    #print(y_test)
    #np.random.seed(0)
    dt_1=DecisionTreeClassifier(random_state=0)#, max_depth=2)
    dt_1=dt_1.fit(X_train,y_train)
    y_pred=dt_1.predict(X_test)
    accuracy=(metrics.accuracy_score(y_test,y_pred)*100)
    #print("Accuracy: ",metrics.accuracy_score(y_test,y_pred)*100)
    height+=(1+dt_1.tree_.max_depth)
    print(height)
    feature_importances = pd.DataFrame(dt_1.feature_importances_,index = X_train.columns,columns=['importance'])#.sort_values('importance', ascending=False)
    #print(feature_importances)
    #ctf=ct
    #rown=2
    #sheet3.write(ctf,0,puma)
    #sheet3.write(ctf,1,race)
    '''
    for row in feature_importances.index:
      sheet3.write(ctf,rown,feature_importances['importance'][row])
      rown+=1
    '''
    #feature_importances=np.array(feature_importances['importance'])
    features_used=(len(feature_importances[feature_importances['importance']>0]))
    sheet2.write(ct,2,accuracy)
    sheet2.write(ct,3,height)
    sheet2.write(ct,10,features_used)
    for rown in range(20):
      sheet3.write(ct,rown+2,feature_importances['importance'][rown])
    #accuracy for blacks
    #y_black_test=y_test[X_test['RACE']==2]
    #print(len(y_black_test))
    #X_black_test=X_test[X_test['RACE']==2]
    #print(len(X_black_test))
    #y_black_pred=dt_1.predict(X_black_test)
    #print('%age of Black People:',len(puma_1_df[(puma_1_df['RACE']==2)].index)/len(puma_1_df)*100)# & (puma_1_df['RACE']==2)]
    #print('%age of Black Women:',len(puma_1_df[(puma_1_df['RACE']==2)& (puma_1_df['SEX']==2)].index)/len(puma_1_df)*100)
    #print('%age of Non-White People:',len(puma_1_df[puma_1_df['RACE']!=1].index)/len(puma_1_df)*100)
    #print('%age of Women:',len(puma_1_df[puma_1_df['SEX']==2].index)/len(puma_1_df)*100)
    #print('%age of People > 60 yrs:',len(puma_1_df[(puma_1_df['AGE']>60)].index)/len(puma_1_df)*100)
    #print("Percentage = ",(len(puma_1_df_black.index)/len(puma_1_df)*100))

    max_d=dt_1.tree_.max_depth
    acc_list=[]
    for i in range(1,max_d+1):
      np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0, max_depth=i)
      dt_1=dt_1.fit(X_train,y_train)
      y_pred=dt_1.predict(X_test)
      acc_list.append(metrics.accuracy_score(y_test,y_pred)*100)
    try:
      sheet2.write(ct,4,2+acc_list.index(max(acc_list)))
      #print("Optimal Tree Height:",2+acc_list.index(max(acc_list)))
      #print("Max Accuracy:",max(acc_list))
      sheet2.write(ct,5,max(acc_list))
      np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0, max_depth=9)
      dt_1=dt_1.fit(X_train,y_train)
      #print(1+dt_1.tree_.max_depth)
      y_pred=dt_1.predict(X_test)
      sheet2.write(ct,6,(metrics.accuracy_score(y_test,y_pred)*100))
      #print('Accuracy for Max Height 10:',(metrics.accuracy_score(y_test,y_pred)*100))
      np.random.seed(0)
      dt_1=DecisionTreeClassifier(random_state=0, max_depth=19)
      dt_1=dt_1.fit(X_train,y_train)
      #print(1+dt_1.tree_.max_depth)
      y_pred=dt_1.predict(X_test)
      sheet2.write(ct,7,(metrics.accuracy_score(y_test,y_pred)*100))
      #print('Accuracy for Max Height 20:',(metrics.accuracy_score(y_test,y_pred)*100))
      sheet2.write(ct,8,round((len(df1[(df1['PUMA']==puma)&(df1['RACE']==race)])/len(df1[df1['PUMA']==puma]))*100,3))
    except:
      sheet2.write(ct,4,'Uh oh')
      continue
    #print("Accuracy for black rows is ",metrics.accuracy_score(y_black_test,y_black_pred)*100)
    #tree.plot_tree(dt_1);
wbb.save('RMforDTdedicated.xls')

#Random Merging

In [ ]:
wbb=Workbook()

In [ ]:
#prepare hist
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import pipeline_dp as pdp
import math
import random
import data_tools_syr3 as syr
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics, tree 
from scipy import stats
import timeit
import xlwt
from xlwt import Workbook
def prep_hist(df):
  mar_df_1=df.groupby(['PUMA','AGE','RACE','SEX','OWNERSHP','INCTOT_DECILE'])[['GQ']]
  hist=mar_df_1.count()
  hist.reset_index()
  hist1=hist.copy()
  for i in range(6):
    hist1=hist1.reset_index(level=0)
  #print(hist1)
  hist1=hist1.rename(columns={'GQ':"COUNT"})
  return hist1
def prep_hist2(df,fnames):
  mar_df_1=df.groupby(fnames)[['INDNAICS']]
  hist=mar_df_1.count()
  hist.reset_index()
  hist1=hist.copy()
  for i in range(len(fnames)):
    hist1=hist1.reset_index(level=0)
  #print(hist1)
  hist1=hist1.rename(columns={'INDNAICS':"COUNT"})
  return hist1
def prep_hist3(df,fnames,cntname):
  mar_df_1=df.groupby(fnames)[[cntname]]
  hist=mar_df_1.count()
  hist.reset_index()
  hist1=hist.copy()
  for i in range(len(fnames)):
    hist1=hist1.reset_index(level=0)
  #print(hist1)
  hist1=hist1.rename(columns={cntname:"COUNT"})
  return hist1
#create function to replicate rows based on GQ value
def count_scale_df(hist):
  hist_r=hist.copy()
  for index, row in hist.iterrows():
    #print(row['COUNT'])
    for i in range(int(row['COUNT'])):
      hist_r=hist_r.append(row)
  return hist_r
#
def count_scale_df2(hist):
  hist_r=hist.copy()
  concat_rows=[]
  for index, row in hist.iterrows():
    #print(row['COUNT'])
    for i in range(int(row['COUNT'])):
      concat_rows.append(row)
  hist_r=pd.concat(concat_rows)
  return hist_r

def redact_counts_df(hist,k): #Outputs redact-suppressed histogram
  hist1=hist.copy()
  counts_1=list(hist1['COUNT'])
  print("counts:",counts_1)
  #print("Hola:",counts_1)
  for i in range(len(counts_1)):
    if counts_1[i]<k:
      counts_1[i]=(k//2)
  hist1['COUNT']=counts_1
  return hist1
#
def dp_counts_df(hist,eps): #Outputs redact-suppressed histogram
  hist1=hist.copy()
  counts_1=list(hist1['COUNT'])
  #try:
    #dp_counts=add_dp_hist(counts_1,eps)
  #except:
  dp_counts=syr.add_dp_hist(counts_1,eps)
  hist1['COUNT']=dp_counts
  return hist1
#
def randmerge_counts_df(hist,k): #Outputs redact-suppressed histogram
  hist1=hist.copy()
  #start=timeit.default_timer()
  counts_1=list(hist1['COUNT'])
  #try:
  dp_counts=syr.spread_k_anonymise(counts_1,k)
  #except:
    #dp_counts=syr.spread_k_anonymise(counts_1,k)
  hist1['COUNT']=dp_counts
  #stop=timeit.default_timer()
  #print('Time: ',stop-start)
  return hist1

def adjmerge_counts_df(hist,k):
  #print('hey')
  hist1=hist.copy()
  #start=timeit.default_timer()
  counts_1=list(hist1['COUNT'])
  #try:
  dp_counts=syr.adj_k_anonymise(counts_1,k)
  #except:
    #dp_counts=syr.spread_k_anonymise(counts_1,k)
  hist1['COUNT']=dp_counts
  #stop=timeit.default_timer()
  #print('Time: ',stop-start)
  return hist1
#Get Randmerge Stats
def dedicated_dt_randmerge(dfname,shortdfname,k,wbb): #init wbb=Workbook() prior to running
  df1=pd.read_csv(dfname)#"tx_acs_2019.csv")
  pumas=list(df1['PUMA'].unique())
  ct=0
  #k=6
  sheet2=wbb.add_sheet(str(shortdfname)+' - RM '+str(k))
  sheet3=wbb.add_sheet(str(shortdfname)+' Feat Imp - RM '+str(k))
  sheet2.write(0, 0, 'PUMAs')
  sheet2.write(0, 1, 'Race')
  sheet2.write(0, 2, 'Accuracy')
  sheet2.write(0, 3, 'Height')
  sheet2.write(0, 4, 'Opt Ht')
  sheet2.write(0, 5, 'Opt Acc')
  sheet2.write(0, 6, 'Max Ht 10')
  sheet2.write(0, 7, 'Max Ht 20')
  sheet2.write(0, 8, 'Pop %')
  sheet3.write(0,0,'PUMAs')
  sheet3.write(0,1,'Race')
  sheet3.write(0,2,'GQ')
  sheet3.write(0,3,'OWNERSHP')
  sheet3.write(0,4,'FAMSIZE')
  sheet3.write(0,5,'NCHILD')
  sheet3.write(0,6,'AGE')
  sheet3.write(0,7,'SEX')
  sheet3.write(0,8,'MARST')
  sheet3.write(0,9,'RACE')
  sheet3.write(0,10,'HISPAN')
  sheet3.write(0,11,'EDUC')
  sheet3.write(0,12,'SECTOR')
  sheet3.write(0,13,'POVERTY')
  sheet3.write(0,14,'DENSITY')
  sheet3.write(0,15,'VETDISAB')
  sheet3.write(0,16,'DIFFREM')
  sheet3.write(0,17,'DIFFPHYS')
  sheet3.write(0,18,'DIFFEYE')
  sheet3.write(0,19,'DIFFHEAR')
  sheet3.write(0,20,'HHWT')
  sheet3.write(0,21,'PERWT')
  #pumas=[pumas[1]]
  for puma in pumas:
    print('\n',puma)
    for race in range(1,8):
      print('\nRace - ',race)
      ct+=1
      sheet2.write(ct,0,puma)
      sheet2.write(ct,1,race)
      #print('\nRACE = ',race)
      #print(puma_1_df['COUNT'])
      accuracy=0
      height=0
      feature_importances_n=np.zeros(20)
      features_used=0
      #for _ in range(100):
      puma_1_df=df1
      puma_1_df=puma_1_df[(puma_1_df['PUMA']==puma)&(puma_1_df['RACE']==race)]
      #feature_cols=['OWNERSHP','FAMSIZE','NCHILD','AGE','SEX','RACE','EDUC','SECTOR']
      feature_cols=list(puma_1_df.columns)
      #print(feature_cols)
      feature_cols.remove('INCTOT')
      feature_cols.remove('PUMA')
      feature_cols.remove('INDNAICS')
      feature_cols.remove('Unnamed: 0')
      puma_1_df=prep_hist2(puma_1_df,feature_cols)
      if len(puma_1_df)<2:
        continue
      accuracy=0
      height=0
      feature_importances_n=np.zeros(20)
      features_used=0
      acc_10=0
      acc_20=0
      opt_ht=0
      opt_acc=0
      for _ in range(100):
        puma_1_df=randmerge_counts_df(puma_1_df,k)
        puma_1_df=count_scale_df(puma_1_df)
        try:
          feature_cols.remove('INCTOT_DECILE')
        except:
          print(' ')
        #print(puma_1_df.columns)
        X=puma_1_df[feature_cols]
        y=puma_1_df['INCTOT_DECILE']
        #print('\n',feature_cols)
        np.random.seed(0)
        try:
          X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
        except:
          #sheet2.write(ct,2,'Beep')
          #print('Not enough individuals')
          continue
        #print(y_test)
        #np.random.seed(0)
        dt_1=DecisionTreeClassifier(random_state=0)#, max_depth=2)
        dt_1=dt_1.fit(X_train,y_train)
        y_pred=dt_1.predict(X_test)
        accuracy=(metrics.accuracy_score(y_test,y_pred)*100)
        #print("Accuracy: ",metrics.accuracy_score(y_test,y_pred)*100)
        height+=(1+dt_1.tree_.max_depth)
        print(height)
        feature_importances=np.array(pd.DataFrame(dt_1.feature_importances_,index = X_train.columns,columns=['importance']))
        feature_importances_n+=feature_importances['importance']
        max_d=dt_1.tree_.max_depth
        acc_list=[]
        for i in range(1,max_d+1):
          np.random.seed(0)
          dt_1=DecisionTreeClassifier(random_state=0, max_depth=i)
          dt_1=dt_1.fit(X_train,y_train)
          y_pred=dt_1.predict(X_test)
          acc_list.append(metrics.accuracy_score(y_test,y_pred)*100)
        try:
          opt_ht+=(2+acc_list.index(max(acc_list)))
          #print("Optimal Tree Height:",2+acc_list.index(max(acc_list)))
          #print("Max Accuracy:",max(acc_list))
          opt_acc+=(max(acc_list))
          np.random.seed(0)
          dt_1=DecisionTreeClassifier(random_state=0, max_depth=9)
          dt_1=dt_1.fit(X_train,y_train)
          #print(1+dt_1.tree_.max_depth)
          y_pred=dt_1.predict(X_test)
          acc_10+=((metrics.accuracy_score(y_test,y_pred)*100))
          #print('Accuracy for Max Height 10:',(metrics.accuracy_score(y_test,y_pred)*100))
          np.random.seed(0)
          dt_1=DecisionTreeClassifier(random_state=0, max_depth=19)
          dt_1=dt_1.fit(X_train,y_train)
          #print(1+dt_1.tree_.max_depth)
          y_pred=dt_1.predict(X_test)
          acc_20+=((metrics.accuracy_score(y_test,y_pred)*100))
          #print('Accuracy for Max Height 20:',(metrics.accuracy_score(y_test,y_pred)*100))
          #sheet2.write(ct,8,round((len(df1[(df1['PUMA']==puma)&(df1['RACE']==race)])/len(df1[df1['PUMA']==puma]))*100,3))
        except:
          sheet2.write(ct,4,'Uh oh')
          continue
        #print(feature_importances)
        #ctf=ct
        #rown=2
        #sheet3.write(ctf,0,puma)
        #sheet3.write(ctf,1,race)
        '''
        for row in feature_importances.index:
          sheet3.write(ctf,rown,feature_importances['importance'][row])
          rown+=1
        '''
        #feature_importances=np.array(feature_importances['importance'])
        features_used+=(len(feature_importances[feature_importances['importance']>0]))
      feature_importances_n/=100
      features_used/=100
      accuracy/=100
      height/=100
      acc_10/=100
      acc_20/=100
      opt_ht/=100
      opt_acc/=100
      sheet2.write(ct,2,accuracy)
      sheet2.write(ct,3,height)
      sheet2.write(ct,4,opt_ht)
      sheet2.write(ct,5,opt_acc)
      sheet2.write(ct,10,features_used)
      sheet2.write(ct,8,round((len(df1[(df1['PUMA']==puma)&(df1['RACE']==race)])/len(df1[df1['PUMA']==puma]))*100,3))
      sheet2.write(ct,6,acc_10)
      sheet2.write(ct,7,acc_20)
      for rown in range(20):
        sheet3.write(ct,rown+2,feature_importances_n[rown])
        #accuracy for blacks
        #y_black_test=y_test[X_test['RACE']==2]
        #print(len(y_black_test))
        #X_black_test=X_test[X_test['RACE']==2]
        #print(len(X_black_test))
        #y_black_pred=dt_1.predict(X_black_test)
        #print('%age of Black People:',len(puma_1_df[(puma_1_df['RACE']==2)].index)/len(puma_1_df)*100)# & (puma_1_df['RACE']==2)]
        #print('%age of Black Women:',len(puma_1_df[(puma_1_df['RACE']==2)& (puma_1_df['SEX']==2)].index)/len(puma_1_df)*100)
        #print('%age of Non-White People:',len(puma_1_df[puma_1_df['RACE']!=1].index)/len(puma_1_df)*100)
        #print('%age of Women:',len(puma_1_df[puma_1_df['SEX']==2].index)/len(puma_1_df)*100)
        #print('%age of People > 60 yrs:',len(puma_1_df[(puma_1_df['AGE']>60)].index)/len(puma_1_df)*100)
        #print("Percentage = ",(len(puma_1_df_black.index)/len(puma_1_df)*100))
        #print("Accuracy for black rows is ",metrics.accuracy_score(y_black_test,y_black_pred)*100)
        #tree.plot_tree(dt_1);
  wbb.save('RMforDTdedicated1.xls')
def dedicated_dt_randmerge_all():
  wbb=Workbook()
  dedicated_dt_randmerge("tx_acs_2019.csv",'TX',6,wbb)
  dedicated_dt_randmerge("tx_acs_2019.csv",'TX',10,wbb)
  dedicated_dt_randmerge("ma_acs_2019.csv",'MA',6,wbb)
  dedicated_dt_randmerge("ma_acs_2019.csv",'MA',10,wbb)
  dedicated_dt_randmerge("outlier_acs_2019.csv",'Outlier',6,wbb)
  dedicated_dt_randmerge("outlier_acs_2019.csv",'Outlier',10,wbb)

dedicated_dt_randmerge_all()

Streaming output truncated to the last 5000 lines.
960
959
958
957
956
955
954
953
952
951
950
949
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
919
918
917
916
915
914
913
912
910
908
907
906
905
904
903
902
900
899
898
895
894
893
892
890
889
888
887
886
885
883
882
881
879
878
877
876
875
874
873
872
871
870
869
867
866
865
864
863
862
861
860
859
858
857
855
854
852
851
850
849
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
821
820
819
818
817
816
815
814
813
812
811
809
808
807
805
804
803
802
801
800
799
798
797
796
795
794
793
790
789
788
786
785
784
783
782
781
780
779
778
777
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761
760
759
758
757
756
755
754
753
752
751
750
749
748
747
746
744
743
742
741
740
739
738
737
736
735
734
733
732
731
729
728
727
726
725
724
722
721
720
719
718
716
715
714
713
712
711
710
709
708
707
706
705
703
702
701
700
698
697
6

KeyboardInterrupt: ignored